## Reference states

このレッスンでは、変分アルゴリズムをより速く収束させるために、参照状態で系を初期化する方法について学びます。まず、 [*参照状態*](gloss:reference) を手動で構築する方法を学び、次に変分アルゴリズムで使用可能ないくつかの標準的な選択肢を探ります。

![Reference Workflow](images/reference_workflow.png)

## Default state

*参照状態* とは、問題に対する最初の固定開始状態を意味します。参照状態を準備するために、量子回路の開始時に適切なパラメータ化されていないユニタリー回路 $U_R$ を適用し、 $|\rho\rangle = U_R |0\rangle$ となるようにする必要があります。既存の最適解からの推測やデータ点がある場合、それを出発点として使用すると変分アルゴリズムが早く収束する可能性があります。

最も単純な参照状態はデフォルト状態であり、 $n$ 量子ビット回路の開始状態 $|0\rangle^{\otimes n}$ を使用します。 デフォルト状態の場合、ユニタリー演算子 $U_R \equiv I$ が使われます。その単純さゆえに、デフォルト状態は多くのシナリオで使用される有効な参照状態です。

## Classical reference state

Suppose you have a $3$-qubit system and you want to start in the state $|001\rangle$ instead of the default state $|000\rangle$. This is an example of a purely classical reference state, and to construct it, you simply need to apply an [X gate](https://qiskit.org/documentation/stubs/qiskit.circuit.library.XGate.html) to qubit $0$ (following Qiskit's qubit ordering), as $|001\rangle = X_0 |000\rangle$.

In this case, our unitary operator is $U_R \equiv X_0$, which leads to the reference state $|\rho\rangle \equiv |001\rangle$.

In [ ]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(3)
qc.x(0)

qc.draw("mpl")

## Quantum reference state

Suppose you aim to start with a more complex state that involves superposition and/or entanglement, such as $\frac{1}{\sqrt{2}}(|100\rangle+|111\rangle)$.

To obtain this state from $|000\rangle$, one approach is to use a [Hadamard gate](https://qiskit.org/documentation/stubs/qiskit.circuit.library.HGate.html) on qubit $0$ ($H_0$), a [CNOT (CX)](https://qiskit.org/documentation/stubs/qiskit.circuit.library.CXGate.html) gate with qubit $0$ as the control qubit and qubit $1$ as the target qubit ($CNOT_{01}$), and finally an $X$ gate applied to qubit $2$ ($X_2$).

In this scenario, our unitary operator is $U_{R} \equiv X_2CNOT_{01}H_0|000\rangle$, and our reference state is $|\rho\rangle \equiv \frac{1}{\sqrt{2}}(|100\rangle+|111\rangle)$.

In [ ]:
qc = QuantumCircuit(3)
qc.h(0)
qc.cx(0, 1)
qc.x(2)

qc.draw("mpl")

## Constructing Reference States using template circuits

また、複数の調整可能なパラメータやエンタングルメントを簡単に表現できる [`TwoLocal`](https://qiskit.org/documentation/stubs/qiskit.circuit.library.TwoLocal.html) など、様々なテンプレート回路を利用することができます。これらのテンプレート回路については、次回のレッスンで詳しく説明しますが、パラメータをバインドした *場合* 、参照状態に利用することができます。

In [ ]:
from qiskit.circuit.library import TwoLocal
from math import pi

reference_circuit = TwoLocal(2, "rx", "cz", entanglement="linear", reps=1)
theta_list = [pi / 2, pi / 3, pi / 3, pi / 2]

reference_circuit = reference_circuit.bind_parameters(theta_list)

reference_circuit.decompose().draw("mpl")

## Application-specific reference states

### Quantum Chemistry

In Quantum Chemistry, the *Hartree-Fock* state is an approximation of the ground state of an atom or molecule. If our goal is to create a variational algorithm to find the exact ground state, we can use this known classical approximation as a reference state to help our algorithm converge faster.

この例では、2つの空間軌道と2つの電子（1つのアルファ・スピン、1つのベータ・スピン）を持つ問題に対して、次のように電子構造の問題を生成することができます。

In [ ]:
from qiskit_nature.second_q.circuit.library import HartreeFock
from qiskit_nature.second_q.mappers import JordanWignerMapper

num_spatial_orbitals = 2
num_particles = (1, 1)

mapper = JordanWignerMapper()

h2_reference_state = HartreeFock(
    num_spatial_orbitals=num_spatial_orbitals,
    num_particles=num_particles,
    qubit_mapper=mapper,
)

h2_reference_state.decompose().draw("mpl")

また、水素化リチウム（LiH）のような特定の分子を構成することも可能です。

In [ ]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.circuit.library import HartreeFock
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo

molecule = MoleculeInfo(
    # Coordinates in Angstrom
    symbols=["Li", "H"],
    coords=([0.0, 0.0, 0.0], [0.2, 0.0, 0.0]),
    multiplicity=1,  # = 2*spin + 1
    charge=0,
)

driver = PySCFDriver.from_molecule(molecule)
LiH_problem = driver.run()

mapper = JordanWignerMapper()
LiH_reference_state = HartreeFock(
    num_spatial_orbitals=LiH_problem.num_spatial_orbitals,
    num_particles=LiH_problem.num_particles,
    qubit_mapper=mapper,
)

LiH_reference_state.decompose().draw("mpl")

### Quantum Machine Learning

[変分量子分類器（VQC）](https://learn.qiskit.org/course/machine-learning/variational-classification) の文脈では、訓練データは、 *特徴量マップ* として知られるパラメータ化された回路で量子状態に符号化され、各パラメータの値は訓練データセットのデータ点を表します。 [ZZFeatureMap](https://qiskit.org/documentation/stubs/qiskit.circuit.library.ZZFeatureMap.html) は、この特徴量マップにデータ点($x$) を渡すパラメータ化回路の一種です。

In [ ]:
from qiskit.circuit.library import ZZFeatureMap

data = [0.1, 0.2]

zz_feature_map_reference = ZZFeatureMap(feature_dimension=2, reps=2)
zz_feature_map_reference = zz_feature_map_reference.bind_parameters(data)
zz_feature_map_reference.decompose().draw("mpl")

With this lesson, you learned how to initialize your system using:

- Default reference state
- Classical reference states
- Quantum reference states
- Application-specific reference states

このハイレベルな変分ワークロードは次のようになります。

![Reference Circuit](images/reference_circuit.png)

参照状態は固定された最初の出発点であるのに対し、 *変分形式* を使って、変分アルゴリズムが探索するためのパラメータ化された状態の集合を表す *ansatz* を定義することができます。